In [ ]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# %% [markdown]
# https://pramod-atre.medium.com/disparity-map-computation-in-python-and-c-c8113c63d701#:~:text=Depth%20is%20inversely%20proportional%20to,we%20would%20have%20higher%20Z.
# 
# Depth is inversely proportional to disparity, i.e., from the depth estimation equation, we have Z∝1/(xl−xr). As disparity (xl−xr) increases, Z decreases and for lower disparity (xl−xr), we would have higher Z. This is intuitive if you hold your index finger near your eyes and alternate seeing from your left and right eyes. You will notice that your finger jumps a lot in your view compared to other distant objects. We will use a technique called block matching to find the correspondences between pixels in the two images as outlined in². Here is a summary of steps we need for computing disparity map: 

# %% [code]
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import cv2

# output directory:  "../working/FILENAME"

'''
Direction
1. Load left/right images, Create np.array having same dimension, calling it as imgDisparity
2. Declare variables:
    searchWindowSize: also called BlockSize. it defines window size to calculate SSD between img1 and img2.
    maxDisparityRange: also called SearchBlockSize. it defines the rail searchWindow moves on. setup Max range.
3. Define functions: DisparitySSD:
    Might need another functions to find minimum SSD.
    args: img1, img2, searchWindow
    a. define searchWindowSize for applying to general use including edge/corner/boundary cases
    b. define disparityRange (index left and right) for same reason above.
       When doing this, disparity Index Left and right might be varying according to index (y, x), searchWindowSize.
    c. At each index (y,x) --> calculate SSD with index range (defined by c) in img2, which calculate SSDs (defined by b) 
    d. disparity value = y in img1 - y' (index found by SSD) --> Not sure calculate as is or additional operation like abs / square.
'''

def searchWindow(numRow, numCol, y, x, searchWindowSize):
    '''
    making general size confirm considering edge case
    args:
        numRow, numCol: shape of img1 array. they are boundary.
        y, x: row and column of img1
        X/Y searchWindowLength Left/Right/Top/Bottom: block length from center index (y, x) --> for edge cases
    '''
    ySearchWindowLengthTop = ySearchWindowLengthBottom = xSearchWindowLengthLeft = xSearchWindowLengthRight = searchWindowSize // 2
    
    if y - ySearchWindowLengthTop < 0:
        ySearchWindowLengthTop = y
    if y + ySearchWindowLengthBottom > numRow -1:
        ySearchWindowLengthBottom = numRow -1 -y
    if x - xSearchWindowLengthLeft < 0:
        xSearchWindowLengthLeft = x
    if x + xSearchWindowLengthRight > numCol -1:
        xSearchWindowLengthRight = numCol -1 -x
    return xSearchWindowLengthLeft, xSearchWindowLengthRight, ySearchWindowLengthTop, ySearchWindowLengthBottom
    

def disparityRangeCal(numRow, numCol, y, x, xSearchWindowLengthLeft, xSearchWindowLengthRight, disparityRange) -> (int, int):
    '''
    disparity calculation range according to each index
    args:
        numRow, numCol: shape of img1 array. they are boundary.
        y, x: row and column of img1
        disparityRange: adjusted odd value considering center. (this range will search both side)
        xSearchWindowLength Left/Right: To calculate addition.
    return:
        beginning and ending index of moving the center of searchWindow.
    '''
    disparityHalf = disparityRange // 2
    xSearchWindowLengthLeft = xSearchWindowLengthLeft
    xSearchWindowLengthRight = xSearchWindowLengthRight
    
    # Needed to consider two things.
    # 1. disparity range considering index (y, x) before considering searchWindowSize.
    xDisparityLeftIndex = max(x - disparityHalf, 0)
    xDisparityRightIndex = min(x + disparityHalf, numCol-1)

    # 2. Need to consider more according to searchWindowLength Left/Right
    if xDisparityLeftIndex - xSearchWindowLengthLeft < 0:
        xDisparityLeftIndex = xSearchWindowLengthLeft
    if xDisparityRightIndex + xSearchWindowLengthRight > numCol -1:
        xDisparityRightIndex = numCol -1 - xSearchWindowLengthRight    
    
    assert xDisparityLeftIndex <= xDisparityRightIndex, "It can not calculate. Disparity Left index is greater than Right based on the inputs"
        
    return xDisparityLeftIndex, xDisparityRightIndex


def disparityMap(imgLeft, imgRight, searchWindowSize = 1, disparityRange = 10):
    assert searchWindowSize > 0, "searchWindow should be bigger than 0(zero)"
    assert disparityRange > 0, "disparityRange should be bigger than 0(zero)"
    if disparityRange % 2 == 0:
        disparityRange -= 1
    if searchWindowSize % 2 == 0:
        searchWindowSize -= 1
    
    imgLeft = cv2.GaussianBlur(imgLeft,(searchWindowSize,searchWindowSize),0)
    imgRight = cv2.GaussianBlur(imgRight,(searchWindowSize,searchWindowSize),0)
    
    numRow, numCol = imgLeft.shape # because imgLeft is for template of imgDisparity 
    imgDisparity = np.zeros(imgLeft.shape, imgLeft.dtype)
    
    for y in range(numRow):
        for x in range(numCol):
            xLengthLeft, xLengthRight, yLengthTop, yLengthBottom = searchWindow(numRow, numCol, y, x, searchWindowSize)
            xDisparityLeftIndex, xDisparityRightIndex = disparityRangeCal(numRow, numCol, y, x, xLengthLeft, xLengthRight, disparityRange)
            
            arraySSD = np.array([])
            imgLeft_temp = imgLeft[y-yLengthTop:y+yLengthBottom+1, x-xLengthLeft:x+xLengthRight+1]
            for j in range(xDisparityLeftIndex, xDisparityRightIndex+1):
                imgRight_temp = imgRight[y-yLengthTop:y+yLengthBottom+1, j-xLengthLeft:j+xLengthRight+1]
                arraySSD = np.append(arraySSD, np.sum(np.square(imgLeft_temp - imgRight_temp)))
            
            minIdxImg2 = np.argmin(arraySSD)
            minIdxImg2 += xDisparityLeftIndex
            disparityValue = np.abs(x - minIdxImg2)
            imgDisparity[y, x] = disparityValue
    imgDisparity = imgDisparity * 255 // np.max(imgDisparity)
    imgDisparity = imgDisparity.astype('uint8')
    return imgDisparity


# %% [code]

imgLeft = plt.imread("../input/homeassignment4/frameLeftgray.png")
imgLeft.shape, imgLeft.dtype
imgRight = plt.imread("../input/homeassignment4/frameRightgray.png")
imgDisparity1 = disparityMap(imgLeft, imgRight, searchWindowSize = 1, disparityRange = 15)
imgDisparity3 = disparityMap(imgLeft, imgRight, searchWindowSize = 3, disparityRange = 15)
imgDisparity5 = disparityMap(imgLeft, imgRight, searchWindowSize = 5, disparityRange = 15)
imgDisparityControl = plt.imread("../input/homeassignment4/disparityMap_Matlab.png")
plt.figure(figsize=(20, 20))


plt.subplot(2, 2, 1)
plt.title("Disparity: Provided")
# plt.imshow(imgDisparity)
plt.imshow(imgDisparityControl, cmap='gray')

plt.subplot(2, 2, 2)
plt.title("Disparity Map1")
# plt.imshow(imgDisparity)
plt.imshow(imgDisparity1, cmap='gray')

plt.subplot(2, 2, 3)
plt.title("Disparity Map3")
# plt.imshow(imgDisparity)
plt.imshow(imgDisparity3, cmap='gray')

plt.subplot(2, 2, 4)
plt.title("Disparity Map5")
# plt.imshow(imgDisparity)
plt.imshow(imgDisparity5, cmap='gray')

# %% [code]

cv2.imwrite("../working/task1-ImgDisparity_block1_range15_G.jpg", imgDisparity1)
cv2.imwrite("../working/task1-ImgDisparity_block3_range15_G.jpg", imgDisparity3)
cv2.imwrite("../working/task1-ImgDisparity_block5_range15_G.jpg", imgDisparity5)

# %% [code]
